In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Introduction
- We use three Keras functionalities (inherits from keras model) to train, evaluate and pedict from the model that developed
    - Model.fit() - for training
    - Model.evaluate() - to evaluate the model
    - Model.predict() - to infere the model on test example or test dataset
- When we use built in loops for training and evaluation, process will be same for both Seqential and Functional API models

## A First end -to-end Example
- Data can be fed to training loops either using
    - Numpy Arrays (When the data is small and can be fit into memory)
    - tf.data Dataset objects

Lets consider the following model for MNIST classification:

A Typical end-to-end workflow looks like consists of:
- Training
- Validation on Hold out Data (generated from original training data)
- Evaluation on Test Data


In [3]:
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense1")(inputs)
x = layers.Dense(64, activation="relu", name="dense2")(x)
outputs = layers.Dense(10, activation="softmax", name="predictio")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [5]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data
x_train = x_train.reshape(-1,784).astype("float32")/255
x_test = x_test.reshape(-1,784).astype("float32")/255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

# Specify the training configuration (optimizser, loss, metrics)
model.compile(optimizer = keras.optimizers.RMSprop(),
              loss = keras.losses.SparseCategoricalCrossentropy(),
              metrics = [keras.metrics.SparseCategoricalAccuracy()],)

# fit() is for training the model with several parameters
print("Training The Model...")
history = model.fit(x_train, y_train, batch_size=64, epochs=2, validation_data=(x_val, y_val))

Training The Model...
Epoch 1/2
782/782 [==============================] - 3s 3ms/step - loss: 0.3569 - sparse_categorical_accuracy: 0.8988 - val_loss: 0.2033 - val_sparse_categorical_accuracy: 0.9428
Epoch 2/2
782/782 [==============================] - 2s 2ms/step - loss: 0.1684 - sparse_categorical_accuracy: 0.9496 - val_loss: 0.1349 - val_sparse_categorical_accuracy: 0.9605


In [6]:
# history object holds the metrics and losses for each epoch of both training and validation data
history.history

{'loss': [0.35693860054016113, 0.16841818392276764],
 'sparse_categorical_accuracy': [0.8987600207328796, 0.9496200084686279],
 'val_loss': [0.20330537855625153, 0.13489332795143127],
 'val_sparse_categorical_accuracy': [0.942799985408783, 0.9605000019073486]}

In [7]:
# Evaluate the model on the test data using "evaluate" method
print("Evauae on Test Data")
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)
# Get predictions on individual images or batch of images using predict method
print("Generate Predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape", predictions.shape)

Evauae on Test Data
79/79 [==============================] - 0s 2ms/step - loss: 0.1444 - sparse_categorical_accuracy: 0.9568
test loss, test acc: [0.14439605176448822, 0.9567999839782715]
Generate Predictions for 3 samples
1/1 [==============================] - 0s 127ms/step
predictions shape (3, 10)
